In [2]:
import getpass
import os

from dotenv import load_dotenv
load_dotenv(dotenv_path="../../etradingview/settings/.env")  

True

In [3]:
os.environ["LANGSMITH_TRACING"] = "true"

In [4]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

from langchain_openai import OpenAIEmbeddings

text = input("Please type or paste the text to index:\n")
vectorstore_path = "vector_index"
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = splitter.create_documents([text])
vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore.save_local(vectorstore_path)

In [5]:
vectorstore = FAISS.load_local(vectorstore_path, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()

In [20]:

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

user_question = input("What do you want to know?: \n")


retrieved_docs = retriever.invoke(user_question)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])


prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            "You are Saro, a trading assistant. If the user asks a question, "
            "Display images if they are relevant to the question and the context provided."
            "you should answer it based on the context provided below.\n\nContext:\n{context}"
        ),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

In [26]:
import tiktoken

def count_tokens(text: str, model_name: str = "gpt-4o-mini"):
    encoding = tiktoken.encoding_for_model(model_name)
    return len(encoding.encode(text))

In [ ]:
from langchain.callbacks import get_openai_callback
from langchain.callbacks.base import BaseCallbackHandler
from langchain.callbacks.openai_info import OpenAICallbackHandler

from langchain.callbacks.manager import CallbackManager
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI
from typing import Any, Dict


# 1. Define your streaming handler

class StreamHandler(BaseCallbackHandler):
    def __init__(self):
        self.total_tokens = 0
        self.prompt_tokens = 0
        self.completion_tokens = 0

    def on_llm_new_token(self, token: str, **kwargs):
        self.completion_tokens += len(token) 
        self.total_tokens += len(token)
    
        print(token, end='', flush=True)
    
    def on_llm_start(self, serialized: Dict[str, Any], prompts: list[str], **kwargs: Any) -> None:
        # This is where you'd count prompt tokens if your LLM provider
        # doesn't provide it directly in the response metadata
        for prompt in prompts:
            self.prompt_tokens += len(prompt) # Again, using char count as a proxy

            print(prompt)

    def on_llm_end(self, outputs: Dict[str, Any], **kwargs: Any) -> None:
        # After the chain completes, you can access the accumulated token counts
        print(f"\n\nTotal Tokens: {self.total_tokens}")
        print(f"Prompt Tokens: {self.prompt_tokens}")
        print(f"Completion Tokens: {self.completion_tokens}")

# 2. Create callback manager
stream_handler = StreamHandler()

callback_manager = CallbackManager([stream_handler])

# 3. Build streaming LLM
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.5,
    streaming=True,
    callback_manager=callback_manager
)

# 4. Build chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=False,
    combine_docs_chain_kwargs={"prompt": prompt}
)

# 5. Wrap streaming in cost-tracking context
with get_openai_callback() as cb:
    result = qa_chain.invoke({
        "question": user_question,
        "chat_history": [],
        "context": context
    })




System: You are Saro, a trading assistant. If the user asks a question, Display images if they are relevant to the question and the context provided.you should answer it based on the context provided below.

Context:
=> Value: 0 TAKE PROFIT 3 ============= - Take profit 3 => Value: 40 - Value => Value: ATR - ATR period => Value: 5 - Extra => Value: 14 - Partial close => Value: 0 ALERTS ====== - Long => Value: 100 - Short => Value: D=Buy A={{exchange}} V={{qty}} NUM={{No}} - Long exit => Value: D=Sell A={{exchange}} V={{qty}} NUM={{No}} - Short exit => Value: X=Buy A={{exchange}} P={{partial_close}} NUM={{No}} - Always display these results in a clear, well-formatted table. --- **Documentation and Guides** - **Strategy Installation Instructions:** Adding username After selecting your preferred plan , you'll need to provide us with your TradingView username. The video below demonstrates where you can locate your username. [Whatch video](https://isalgotest.s3.amazonaws.com/docs/copy_usern